In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import regex as re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

data = TfidfVectorizer().fit_transform(songs4.text)

X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(data)

In [ ]:
X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
sns.scatterplot(X_embedded[:, 0], X_embedded[:, 1],hue=songs4.index, marker="o")
plt.title("Songtexte")
plt.show()